<a href="https://colab.research.google.com/github/nhonlvsoict/computer-vision-project-20212/blob/master/%5BTraining_Triplet%5D_Facedetect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall -y keras-nightly
!pip install tensorflow==1.15.0
!pip install keras==2.1.5
!pip install h5py==2.10.0
!pip install tensorflow_gpu==1.15.0

Uninstalling keras-nightly-2.5.0.dev2021032900:
  Successfully uninstalled keras-nightly-2.5.0.dev2021032900
     |████████████████████████████████| 412.3MB 27kB/s 
     |████████████████████████████████| 512kB 38.6MB/s 
     |████████████████████████████████| 3.8MB 28.5MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=3ab6169774ba913edc8a1a42c2291541098b33d76c93f8a4faf1086cf1e562f1
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Found existing installation: tensorflow-estimator 

In [ ]:
import numpy as np
import cv2

In [ ]:
import pickle

with open('/content/drive/MyDrive/Colab Notebooks/X_train_tripletCNN.pkl', 'rb') as f:
    X = pickle.load(f)
    X = np.array(X)
    X=X[2400:]
    X = X/255.
with open('/content/drive/MyDrive/Colab Notebooks/y_train_tripletCNN.pkl', 'rb') as f:
    y = pickle.load(f)
    y = np.array(y)
    y=y[2400:]
with open('/content/drive/MyDrive/Colab Notebooks/name_mapCNN.pkl', 'rb') as f:
    name_map = pickle.load(f)
    # name_map=name_map[2400:]

In [ ]:
import gc

gc.collect()


In [ ]:
np.shape(X), np.shape(y), np.shape(name_map)


In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import applications

In [ ]:

import tensorflow as tf
from tensorflow.keras import backend as K
import h5py
import tensorflow.keras.applications.vgg16 as vgg16

def triplet_model():
    #first branch
    vgg_model = vgg16.VGG16(weights=None, include_top=False, input_shape=(221, 221, 3))
    first_vgg = vgg_model.output
    first_vgg = GlobalAveragePooling2D()(first_vgg)
    first_vgg = Dense(4096, activation='relu')(first_vgg)
    first_vgg = Dropout(0.6)(first_vgg)
    first_vgg = Dense(4096, activation='relu')(first_vgg)
    first_vgg = Dropout(0.6)(first_vgg)
    first_vgg = Lambda(lambda x: K.l2_normalize(x,axis=1))(first_vgg)
    first_branch = Model(inputs=vgg_model.input, outputs=first_vgg)

    #second branch
    second_branch = Input(shape=(221, 221, 3))
    second_conv = Conv2D(96, kernel_size=(8,8), strides=(16,16), padding='same')(second_branch)
    second_max = MaxPool2D(pool_size=(3,3), strides=(2,2), padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(second_max)

    #third branch
    third_branch = Input(shape=(221, 221, 3))
    third_conv = Conv2D(96, kernel_size=(8,8), strides=(32,32), padding='same')(third_branch)
    third_max = MaxPool2D(pool_size=(7,7), strides=(4,4), padding='same')(third_conv)
    third_max = Flatten()(third_max)
    third_max = Lambda(lambda x: K.l2_normalize(x, axis=1))(third_max)
                       
    merge_one = concatenate([second_max, third_max])
    merge_two = concatenate([merge_one, first_branch.output])
    emb = Dense(4096)(merge_two)
    emb = Dense(128)(emb)
    l2_norm_final = Lambda(lambda x: K.l2_normalize(x, axis=1))(emb)
                        
    final_model = Model(inputs=[second_branch, third_branch, first_branch.input], outputs=l2_norm_final)

    return final_model

In [ ]:
triplet_model = triplet_model()
triplet_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 221, 221, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 221, 221, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 221, 221, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 110, 110, 64) 0           block1_conv2[0][0]               
___________

In [ ]:
batch_size = 24

_EPSILON = K.epsilon()
def triplet_loss(y_true, y_pred):
    y_pred = K.clip(y_pred, _EPSILON, 1.0 - _EPSILON)
    loss = 0.
    g = 1.
    for i in range(0, batch_size, 3):
        try:
            q_embedding = y_pred[i]
            p_embedding = y_pred[i+1]
            n_embedding = y_pred[i+2]
            D_q_p = K.sqrt(K.sum((q_embedding - p_embedding)**2))
            D_q_n = K.sqrt(K.sum((q_embedding - n_embedding)**2))
            loss = loss + g + D_q_p - D_q_n
        except:
            continue
    loss = loss/batch_size*3
    return K.maximum(loss, 0)

In [ ]:
triplet_model.compile(loss=triplet_loss, optimizer=SGD(lr=0.001, momentum=0.9, nesterov=True))

In [ ]:
def image_batch_generator(images, labels, batch_size):
    labels = np.array(labels)
    while True:
        batch_paths = np.random.choice(a = len(images), size = batch_size//3)
        input_1 = []
        
        for i in batch_paths:
            p = np.where(labels == labels[i])[0]
            n = np.where(labels != labels[i])[0]
            
            j = np.random.choice(p)
            while j == i:
                j = np.random.choice(p)
             
            k = np.random.choice(n)
            while k == i:
                k = np.random.choice(n)
            
            input_1.append(images[i])
            input_1.append(images[j])
            input_1.append(images[k])

        input_1 = np.array(input_1)
        input = [input_1, input_1, input_1]
        yield(input, np.zeros((batch_size, )))

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = '/content/drive/MyDrive/Colab Notebooks/triplet_weight0.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_weights_only=True, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
from tensorflow.keras.models import load_model

print("Loading pre-trained weight")
triplet_model.load_weights('/content/drive/MyDrive/Colab Notebooks/triplet_weight0.hdf5')

Loading pre-trained weight


In [ ]:
# training modal
triplet_model.fit_generator(generator=image_batch_generator(X, y, batch_size),
                   steps_per_epoch=len(X)//batch_size,
                   epochs=2000,
                   verbose=1,
                   initial_epoch= 1463,
                   callbacks=callbacks_list)
     

Epoch 1464/2000
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
240/241 [============================>.] - ETA: 0s - loss: 0.1779
Epoch 01464: loss improved from inf to 0.17714, saving model to /content/drive/MyDrive/Colab Notebooks/triplet_weight0.hdf5
241/241 [==============================] - 212s 880ms/step - loss: 0.1771
Epoch 1465/2000
240/241 [============================>.] - ETA: 0s - loss: 0.1731
Epoch 01465: loss improved from 0.17714 to 0.17392, saving model to /content/drive/MyDrive/Colab Notebooks/triplet_weight0.hdf5
241/241 [==============================] - 199s 826ms/step - loss: 0.1739
Epoch 1466/2000
240/241 [============================>.] - ETA: 0s - loss: 0.1597
Epoch 01466: loss improved from 0.17392 to 0.15899, saving model to /content/drive/MyDrive/Colab Notebooks/triplet_weight0.hdf5
241/241 [==============================] - 198s 822ms/step - loss: 0.1590
Epoch 1467/2000
240/241 [============================>.] -

In [ ]:
#save 128 dim vector to array
from tqdm import tqdm
embs128 = []
for x in tqdm(X):
    image = x/255.
    image = np.expand_dims(image, axis=0)
    emb128 = triplet_model.predict([image, image, image])
    embs128.append(emb128[0])
    del image

embs128 = np.array(embs128)

In [ ]:
np.shape(embs128)

In [ ]:

with open("/content/drive/MyDrive/Colab Notebooks/embs128.pkl", "wb") as f:
    pickle.dump(embs128, f)

In [ ]:
#Create LFW_128_labels.tsv
with open('/content/drive/MyDrive/Colab Notebooks/LFW_128_labels.tsv', 'w') as f:
    for label in y:
        l = str(name_map[label])
        f.write(l + '\n')

In [ ]:
#test modal
with open('/content/drive/MyDrive/Colab Notebooks/name_map1.pkl', 'rb') as f:
    names = pickle.load(f)
    names = np.array(names)
    # names=names[2400:]
np.shape(names)    

In [ ]:
#Load all vector embedding LFW of my model
with open('/content/drive/MyDrive/Colab Notebooks/embs128.pkl', 'rb') as f:
    embs128 = pickle.load(f)
    embs128 = np.array(embs128)
with open('/content/drive/MyDrive/Colab Notebooks/LFW_128_labels.tsv', 'r') as f:
    names_embs128 = f.readlines()

In [ ]:
import time
import dlib
import cv2
import numpy as np
from google.colab.patches import cv2_imshow


In [ ]:
cnn_face_detector = dlib.cnn_face_detection_model_v1('/content/drive/MyDrive/Colab Notebooks/mmod_human_face_detector.dat')

In [ ]:

# image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/lfw/Yoko_Ono/Yoko_Ono_0002.jpg')
image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/sample.jpg')

start = time.time()
# Khai báo việc sử dụng các hàm của dlib
hog_face_detector = dlib.get_frontal_face_detector()

faces_hog = hog_face_detector(image, 1)

for face in faces_hog:
    x = face.left()
    y = face.top()
    w = face.right() - x
    h = face.bottom() - y
    frame = image[y:y+h, x:x+w]
    frame = cv2.resize(frame, (221, 221))
    frame = frame /255.
    frame = np.expand_dims(frame, axis=0)
    emb128 = triplet_model.predict([frame, frame, frame])
    
    minimum = 99999
    person = -1
    for k, e in enumerate(embs128):
        #Euler distance
        dist = np.linalg.norm(emb128-e)
        if dist < minimum:
            minimum = dist
            person = k
    end = time.time()
    
    cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), 2)
    cv2.putText(image, str(names_embs128[person])[0:-1], (x - 10, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
print("HOG + SVM Execution time: " + str(end-start))
cv2_imshow(image)

In [ ]:
from tensorflow.keras.models import load_model

print("Loading pre-trained weight for CNN")
triplet_model.load_weights('/content/drive/MyDrive/Colab Notebooks/triplet_weight0.hdf5')

In [ ]:

image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/sample.jpg')


# Thực hiện xác định bằng CNN
start = time.time()
faces_cnn = cnn_face_detector(image, 1)


for face in faces_cnn:
    x = face.rect.left()
    y = face.rect.top()
    w = face.rect.right() - x
    h = face.rect.bottom() - y
    frame = image[y:y+h, x:x+w]
    frame = cv2.resize(frame, (221, 221))
    frame = frame /255.
    frame = np.expand_dims(frame, axis=0)
    emb128 = triplet_model.predict([frame, frame, frame])
    minimum = 99999
    person = -1
    for k, e in enumerate(embs128):
        #Euler distance
        dist = np.linalg.norm(emb128-e)
        if dist < minimum:
            minimum = dist
            person = k
            # print(dist)
    end = time.time()

    cv2.rectangle(image, (x,y), (x+w,y+h), (0,0,255), 2)
    cv2.putText(image, str(names_embs128[person])[0:-1], (x - 10, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
print("CNN Execution time: " + str(end-start))
cv2_imshow(image)